In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the "../input/" directory.
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('./dataset'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import os
# import jieba
import swifter
import numpy as np
import pandas as pd

from tqdm import tqdm
# from pypinyin import lazy_pinyin, Style, load_single_dict, load_phrases_dict

D:\Softwares\Anaconda3\envs\test\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [11]:
style = Style.TONE3

NameError: name 'Style' is not defined

In [3]:
with open("./user_dict.txt", "w", encoding="utf8") as fw:
    fw.write("""[*] 4
[LAUGH] 4
[SONANT] 4
[ENs] 4
[MUSIC] 4
君意 4
+ 10""")
jieba.load_userdict("user_dict.txt")

# import shutil
# shutil.copytree("../input/dpcode0/DeepSpeechRecognition-master/", "./DeepSpeechRecognition-master/")

# os.listdir("./DeepSpeechRecognition-master/")

dev = pd.read_csv("./dataset/dev.csv")
test = pd.read_csv("./dataset/test.csv")
train = pd.read_csv("./dataset/train.csv")

# train.words.value_counts()[:20]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lizeda\AppData\Local\Temp\jieba.cache
Loading model cost 1.019 seconds.
Prefix dict has been built successfully.


In [ ]:
len([i for i in dev.words if "+" in i])

In [4]:
def lcut(x):
    if x.startswith("["):
        return [x]
    return jieba.lcut(x)

In [5]:
en2cn = {'A': '诶','B': '必','C': '细','D': '地','E': '亿','F': 'f','G': '计',
'H': 'h','I': '爱','J': '戒','K': '尅','L': 'l','M': 'm','N': '恩','O': '欧','P': '批','Q': 'q',
'R': '儿','S': 's','T': '剃','U': '优','V': '微','W': 'z','X': 'x','Y': '外','Z': 'z'}

en2cn_dict = {}
for i in en2cn:
    en2cn_dict[i] = lazy_pinyin(en2cn[i], style=style)[0]
    en2cn_dict[i.lower()] = en2cn_dict[i]
# load_single_dict(en2cn_dict)

In [6]:
def errorpy(words):
    words = words.strip()
    if words.isalpha() and len(words)==1:
        return en2cn_dict[words]
    elif len(words)>1:
        print(words)
        return [en2cn_dict[i]  if i.isalpha() and len(i)==1 else i for i in words.split(" ")]                  
    else:
        return words

In [7]:
print(lazy_pinyin('12个[UNK]的 小app兔子 r n g im a teacher ', style=style, errors=errorpy))

12
[UNK]
app
r n g im a teacher
['12', 'ge4', '[UNK]', 'de', 'xiao3', 'app', 'tu4', 'zi', 'er2', 'en1', 'ji4', 'im', 'ei2', 'teacher']


In [9]:
import string, re
# punctuation = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＝＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."""
punctuation = '!"#$%&\'()*+,-./:;=?@\\^_`{|}~' + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＝＠＼＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."""

In [23]:
read_files = ["dev.txt"]#["dev.csv"]#   #["train.csv"] #
wav_lst = []
pny_lst = []
han_lst = []
for file in read_files:
    if "csv" in file:
        data = pd.read_csv("./dataset/"+file)
        data = data.loc[~data.words.isin(['+', '[SONANT]', '[ENS]', '[*]', '[LAUGH]', '[MUSIC]'])]
        data.words = data.words.apply(lambda x:re.sub("[{}]+".format(punctuation), " ", x).strip() )
        data["wav_file"] = data["Unnamed: 0"].swifter.apply(lambda x: "./dataset/train_bytes/wave_"+str(x)+".csv")
        data["pny"] = data["words"].swifter.apply(lambda x: lazy_pinyin(x, style=style, errors=errorpy))
    else:
        print('load ', file, ' data...')
        sub_file = './' + file
        with open(sub_file, 'r', encoding='utf8') as f:
            data = f.readlines()
        for line in tqdm(data):
            wav_file, pny, han = line.split('\t')
            wav_lst.append(wav_file)
            pny_lst.append(pny.split(' '))
            han_lst.append(han.strip('\n'))

load  dev.txt  data...


100%|██████████████████████████████████████████████████████████████████████████| 6086/6086 [00:00<00:00, 338997.80it/s]


In [30]:
def add_pinyin(data):
    data = data.loc[~data.words.isin(['+', '[SONANT]', '[ENS]', '[*]', '[LAUGH]', '[MUSIC]'])]
    data.loc[:, "words"] = data.words.swifter.apply(lambda x:re.sub("[{}]+".format(punctuation), " ", x).strip() )
    data["wav_file"] = data["Unnamed: 0"].swifter.apply(lambda x: "./dataset/train_bytes/wave_"+str(x)+".bin")
    data["pny"] = data["words"].swifter.apply(lambda x: lazy_pinyin(x, style=style, errors=errorpy))
    return data

In [31]:
train_df = add_pinyin(train)
dev_df = add_pinyin(dev)

D:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


app
olay
olay
p k
m a c
b e t
a j
a d
a d
a d
z k
mvp
mvp
mvp
m v p
low
m v p
pk
nba
nba
logo
nba
ok
nba
nba
nba
nba
nba
cba
mvp
mvp
mvp
mvp
nba
nba
nba
cba
mvp
cba
nba
nba
nba
cba
nba
cba
nba
nba
cba
cba
mvp
nba
cba
A A
D N A
R N A
qq
vip
n b a
n b a
n b a
nba
nba
nba
nba
nba
nba
nba
vip
vip
buff
buff
buff
buff
solo
solo
solo
adc
adc
q q
q q
q q
o k
pk
pk
man
out
see sky
papi
papi
q q
q q
y s l
y s l
iphone x iphone x s
iphone
iPhone
cctv
running
gai
rapper
freestyle
flow
popping
old school
pk
pk
popping
tv
buff
buff
buff
buff
buff
buff
double kill
biu
biu
w s a d
w s a d
oppo
oppo
buff
boss
man
exo
exo
exo
exo
m m
exo
exo
exo
exo
tfboys
nine
snh
b b b b
miniso
b b
b b
c c
b b
b b
b b
b b
kiko
t f
t f
a j
a j
a j
t f
clean
c l e a n
bb
c c
cc
w i s
v i s
m a c
tfboys
cosplay
cosplay
s n h four eight
s n h four eight
cosplay
mv
mv
boss
boss
hi
app
exo
mike
man
t f boys
t f boys
t f boys
video
angelababy
a b
angelababy
iPhone x
broke
a c c a
a c c a
exo
exo exo
exo
exo
kbs
part
kbs
mons

D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fish
dolphin
monkey
panda
rabbit
elephant
tiger
lion
alligator
Giraffe
Giraffe
nfc
a i
ai
ai
ai
pro
z z
bose
jbl
bose
u e
m p
m p
m p
m p
hifi
hifi
hifi
m p
c d
c d
m p
c d
hifi
hifi
c d
c d
c d
c d
m p
a p e
f l a c
jbl
jbl
jbl
hdr
tcl
iphone
oppo
vivo
oppo find
vivo find
vivo
nex nex
vivo
vivo
vivo
google
google
lg
tcl
tcl htc
google pixel
google pixel
google
nexus
ipad
c c
c c
java
d h a
d h a
d n a
v c
v c
v c
v c
v c
hello
hello
ok
hello
hello
OK
OK
OK
ok
ok
m v p
i t
i t
XFun
ok
ok
r n g
p p k
c s
c s c f
c f
qq
[UNK]
c p
c p
[UNK]
d i y
over
ip



In [32]:
train_df.pny = train_df.pny.swifter.apply(lambda x:" ".join(x))
dev_df.pny = dev_df.pny.swifter.apply(lambda x:" ".join(x))

D:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [41]:
dev_df.wav_file = dev_df.apply(lambda x: "./dataset/dev(SPK)/dev_byte(%s)/wave_"%(x["speaker"])+str(x["Unnamed: 0"])+".bin", axis=1)

D:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [33]:
train_df.loc[:, ["wav_file", "pny", "words"]].to_csv("train.txt", index=False, header=False, sep="\t")

In [45]:
dev_df.loc[~dev_df.speaker.isna(), ["wav_file", "pny", "words"]].to_csv("dev.txt", index=False, header=False, sep="\t")

In [2]:
from utils import compute_fbank, multi_task, generate_fbank

import time
import multiprocessing as mlp

In [20]:
train_df = pd.read_csv("./dataset/dev.txt", delimiter="\t", header=None)

In [21]:
train_df.columns = ["wav_file", "pny", "words"]
train_df.head()

,wav_file,pny,words
0,./dataset/dev(SPK)/dev_byte(SPK001)/wave_1.bin,niu1 niu1 jin1 tian1 wo3 men liao2 shen2 me ne,妞妞 今天我们聊什么呢
1,./dataset/dev(SPK)/dev_byte(SPK002)/wave_2.bin,wo3 men jin1 tian1 liao2 dong4 wu4,我们今天聊动物
2,./dataset/dev(SPK)/dev_byte(SPK001)/wave_3.bin,hao3 ba,好吧
3,./dataset/dev(SPK)/dev_byte(SPK001)/wave_5.bin,na4 ni3 shuo1 yi1 shuo1 dong4 wu4 ba,那你说一说动物吧
4,./dataset/dev(SPK)/dev_byte(SPK002)/wave_6.bin,a1 yi2 ni3 xi3 huan1 shen2 me,阿姨你喜欢什么


In [9]:
for i in os.listdir("./dataset/train_bytes/"):
    if "npz" not in i and "bin" not in i:
        _path = os.path.join("./dataset/train_bytes/", i)
        os.rename(_path, _path + ".bin")

In [40]:
import shutil

In [46]:
flist = []
for _root, _dir, _file in os.walk("dataset/dev(SPK)/", topdown=True):
    for _f in _file:
        if "npz" in _f :
            if _f in flist:
                print(os.path.join(_root, _f))
            else:
                flist.append(_f)
                shutil.copy(os.path.join(_root, _f), os.path.join("./dataset/dev_SPK", _f.split(".npz")[0] + "_" + _root[-7:-1] + "_SPK.npz"))
#             print(os.path.join(_root, _f))
#             os.remove(os.path.join(_root, _f))
#             flist.append(generate_fbank(os.path.join(_root, _f.split("npz")[0]+"bin")))

In [19]:
flist = []
for _root, _dir, _file in os.walk("dataset/train_bytes/", topdown=True):
    for _f in tqdm(_file):
        if "npz" in _f:
            try:
                _data = np.load(os.path.join(_root, _f))["data"]
            except Exception as e:
                print(_f, "Error:", e)
                flist.append((_root, _f))
#                 os.remove(os.path.join(_root, _f))
#                 flist.append(generate_fbank(os.path.join(_root, _f.split("npz")[0]+"bin")))

100%|█████████████████████████████████████████████████████████████████████████| 272652/272652 [06:00<00:00, 755.48it/s]


In [4]:
flist = [('dataset/train_bytes/', 'wave_102969.npz'),
 ('dataset/train_bytes/', 'wave_105297.npz'),
 ('dataset/train_bytes/', 'wave_106764.npz'),
 ('dataset/train_bytes/', 'wave_108128.npz'),
 ('dataset/train_bytes/', 'wave_110.npz'),
 ('dataset/train_bytes/', 'wave_110956.npz'),
 ('dataset/train_bytes/', 'wave_11102.npz'),
 ('dataset/train_bytes/', 'wave_113768.npz'),
 ('dataset/train_bytes/', 'wave_115298.npz'),
 ('dataset/train_bytes/', 'wave_123634.npz'),
 ('dataset/train_bytes/', 'wave_126319.npz'),
 ('dataset/train_bytes/', 'wave_126363.npz'),
 ('dataset/train_bytes/', 'wave_133626.npz'),
 ('dataset/train_bytes/', 'wave_133680.npz'),
 ('dataset/train_bytes/', 'wave_135612.npz'),
 ('dataset/train_bytes/', 'wave_138549.npz'),
 ('dataset/train_bytes/', 'wave_141274.npz'),
 ('dataset/train_bytes/', 'wave_141334.npz'),
 ('dataset/train_bytes/', 'wave_143261.npz'),
 ('dataset/train_bytes/', 'wave_145046.npz'),
 ('dataset/train_bytes/', 'wave_145075.npz'),
 ('dataset/train_bytes/', 'wave_149217.npz'),
 ('dataset/train_bytes/', 'wave_15133.npz'),
 ('dataset/train_bytes/', 'wave_15243.npz'),
 ('dataset/train_bytes/', 'wave_152690.npz'),
 ('dataset/train_bytes/', 'wave_22883.npz'),
 ('dataset/train_bytes/', 'wave_26582.npz'),
 ('dataset/train_bytes/', 'wave_32132.npz'),
 ('dataset/train_bytes/', 'wave_3802.npz'),
 ('dataset/train_bytes/', 'wave_38460.npz'),
 ('dataset/train_bytes/', 'wave_3854.npz'),
 ('dataset/train_bytes/', 'wave_42937.npz'),
 ('dataset/train_bytes/', 'wave_4542.npz'),
 ('dataset/train_bytes/', 'wave_46878.npz'),
 ('dataset/train_bytes/', 'wave_50378.npz'),
 ('dataset/train_bytes/', 'wave_50527.npz'),
 ('dataset/train_bytes/', 'wave_53976.npz'),
 ('dataset/train_bytes/', 'wave_58164.npz'),
 ('dataset/train_bytes/', 'wave_58211.npz'),
 ('dataset/train_bytes/', 'wave_59256.npz'),
 ('dataset/train_bytes/', 'wave_62318.npz'),
 ('dataset/train_bytes/', 'wave_62338.npz'),
 ('dataset/train_bytes/', 'wave_66259.npz'),
 ('dataset/train_bytes/', 'wave_66331.npz'),
 ('dataset/train_bytes/', 'wave_67304.npz'),
 ('dataset/train_bytes/', 'wave_70820.npz'),
 ('dataset/train_bytes/', 'wave_7221.npz'),
 ('dataset/train_bytes/', 'wave_7235.npz'),
 ('dataset/train_bytes/', 'wave_73675.npz'),
 ('dataset/train_bytes/', 'wave_74793.npz'),
 ('dataset/train_bytes/', 'wave_81724.npz'),
 ('dataset/train_bytes/', 'wave_85849.npz'),
 ('dataset/train_bytes/', 'wave_90271.npz'),
 ('dataset/train_bytes/', 'wave_94918.npz'),
 ('dataset/train_bytes/', 'wave_98970.npz'),
 ('dataset/train_bytes/', 'wave_99006.npz')]

In [9]:
for _root, _f in flist:
#     os.remove(os.path.join(_root, _f))
    try:
        _data = np.load(os.path.join(_root, _f))["data"]
    except Exception as e:
        print(_f, "Error:", e)
#     generate_fbank(os.path.join(_root, _f.split("npz")[0]+"bin"))

In [32]:
os.path.join(_root, _f)

'dataset/dev(SPK)/dev_byte(SPK060)\\wave_7221.npz'

In [38]:
_f.split(".npz")[0] + "_" + _root[-7:-1] + "_SPK.npz"

'wave_7221_SPK060_SPK.npz'

In [43]:
os.path.join(_root, _f)

'dataset/dev(SPK)/dev_byte(SPK060)\\wave_7221.npz'

In [44]:
shutil.copy(os.path.join(_root, _f), os.path.join("./dataset/dev_SPK", _f.split(".npz")[0] + "_" + _root[-7:-1] + "_SPK.npz"))

'./dataset/dev_SPK\\wave_7221_SPK060_SPK.npz'

In [39]:
os.path.join("./dev_SPK", _f.split(".npz")[0] + "_" + _root[-7:-1] + "_SPK.npz")

'./dev_SPK\\wave_7221_SPK060_SPK.npz'

In [7]:
train_df["fbank_path"] =  train_df['wav_file'].swifter.apply(lambda x: "./dataset/dev_byte(Android)/" + os.path.basename(x))
train_df.loc[:, ["fbank_path", "pny", "words"]].to_csv("dev_device.txt", index=False, header=False, sep="\t", mode="a+")
train_df["fbank_path"] =  train_df['wav_file'].swifter.apply(lambda x: "./dataset/dev_byte(IOS)/" + os.path.basename(x))
train_df.loc[:, ["fbank_path", "pny", "words"]].to_csv("dev_device.txt", index=False, header=False, sep="\t", mode="a+")
train_df["fbank_path"] =  train_df['wav_file'].swifter.apply(lambda x: "./dataset/dev_byte(Recorder)/" + os.path.basename(x))
train_df.loc[:, ["fbank_path", "pny", "words"]].to_csv("dev_device.txt", index=False, header=False, sep="\t", mode="a+")

In [5]:
train_df.wav_file.swifter.apply(lambda x: "./dataset/dev_byte(Android)/" + os.path.basename(x))

0          ./dataset/dev_byte(Android)/wave_1.bin
1          ./dataset/dev_byte(Android)/wave_2.bin
2          ./dataset/dev_byte(Android)/wave_3.bin
3          ./dataset/dev_byte(Android)/wave_5.bin
4          ./dataset/dev_byte(Android)/wave_6.bin
5          ./dataset/dev_byte(Android)/wave_7.bin
6          ./dataset/dev_byte(Android)/wave_9.bin
7         ./dataset/dev_byte(Android)/wave_10.bin
8         ./dataset/dev_byte(Android)/wave_11.bin
9         ./dataset/dev_byte(Android)/wave_13.bin
10        ./dataset/dev_byte(Android)/wave_15.bin
11        ./dataset/dev_byte(Android)/wave_16.bin
12        ./dataset/dev_byte(Android)/wave_17.bin
13        ./dataset/dev_byte(Android)/wave_18.bin
14        ./dataset/dev_byte(Android)/wave_19.bin
15        ./dataset/dev_byte(Android)/wave_21.bin
16        ./dataset/dev_byte(Android)/wave_23.bin
17        ./dataset/dev_byte(Android)/wave_24.bin
18        ./dataset/dev_byte(Android)/wave_25.bin
19        ./dataset/dev_byte(Android)/wave_27.bin


In [ ]:
train_df.to_csv("./")

In [6]:
# def generate_fbank(x):
#     data = compute_fbank(x)
#     path = x.replace(".csv", "")
#     if not os.path.exists(os.path.dirname(path)):
#         print("making dir:", os.path.dirname(path))
#         os.makedirs(os.path.dirname(path))
#     np.savez_compressed(path, data=data)
#     return path

# def multi_task(data, s_type):
#     data["fbank_path"] =  data['wav_file'].swifter.apply(lambda x: generate_fbank(x))
#     return data

train_files = []

train_files = train_df#.loc[~train_df.wav_file.swifter.apply(lambda x: os.path.exists(x.replace("bin", "npz")))]
# train
t0 = time.time()
N = train_files.shape[0]
print(N)
num_cpu = 40 # cpu数量
pool = mlp.Pool(num_cpu)

results = []
batch_num = N // num_cpu
print('batch_num',batch_num)
for i in range(num_cpu):
    if i == num_cpu-1:
        offset = N
    else:
        offset = (i+1)*batch_num
    task_train = train_files.iloc[i*batch_num : offset]
    print('task',i,'shape',len(task_train))
    result = pool.apply_async(multi_task,(task_train, 'train',))
    results.append(result)
pool.close()
pool.join()
print('finished',len(results))

t1 = time.time()
print('time', int(t1 - t0), 's')
trs = pd.DataFrame()
for result in tqdm(results):
    trs = pd.concat([trs, result.get()])
print(trs.shape[0])

6085
batch_num 152
task 0 shape 152
task 1 shape 152
task 2 shape 152
task 3 shape 152
task 4 shape 152
task 5 shape 152
task 6 shape 152
task 7 shape 152
task 8 shape 152
task 9 shape 152
task 10 shape 152
task 11 shape 152
task 12 shape 152
task 13 shape 152
task 14 shape 152
task 15 shape 152
task 16 shape 152
task 17 shape 152
task 18 shape 152
task 19 shape 152
task 20 shape 152
task 21 shape 152
task 22 shape 152
task 23 shape 152
task 24 shape 152
task 25 shape 152
task 26 shape 152
task 27 shape 152
task 28 shape 152
task 29 shape 152
task 30 shape 152
task 31 shape 152
task 32 shape 152
task 33 shape 152
task 34 shape 152
task 35 shape 152
task 36 shape 152
task 37 shape 152
task 38 shape 152
task 39 shape 157
finished 40
time 1056 s


100%|█████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 725.09it/s]


6085


In [ ]:
def mk_am_vocab(data):
    vocab = []
    for line in tqdm(data):
        for pny in line:
            if pny not in vocab:
                vocab.append(pny)
    vocab.append('_')
    return vocab

def mk_lm_pny_vocab(data):
    vocab = ['<PAD>']
    for line in tqdm(data):
        for pny in line:
            if pny not in vocab:
                vocab.append(pny)
    return vocab

In [ ]:
len({j for p in pny_lst for j in p})

In [ ]:
len(mk_am_vocab(pny_lst))

In [ ]:
print('make am vocab...')
am_vocab_tr = mk_am_vocab(train_df.pny)
print('make lm pinyin vocab...')
pny_vocab_tr = mk_lm_pny_vocab(train_df.pny)
print('make am vocab...')
am_vocab_dev = mk_am_vocab(dev_df.pny)
print('make lm pinyin vocab...')
pny_vocab_dev = mk_lm_pny_vocab(dev_df.pny)

In [ ]:
count = 0
for i in set(pny_vocab_tr):
    if i.isalpha():
        print(i)
        count+=1
count

In [ ]:
count = 0
for i in set(pny_vocab_dev):
    if i.isalpha():
        print(i)
        count+=1
count

In [ ]:
alpha_list_tr = []
for i in set(pny_vocab_tr):
    if i.isalpha():
        alpha_list_tr.append(i)
alpha_list_tr

In [ ]:
dev_alpha_list = []
for i in set(pny_vocab_dev):
    if i.isalpha():
        dev_alpha_list.append(i)

In [ ]:
print("全是字母的：", alpha_list_tr)

In [ ]:
print("全是字母的：", dev_alpha_list)

In [ ]:
# 本地中文拼音库
with open("pinyin.txt", "r") as f:
    pinyinLib = f.read().split("\n")

en_list = []
for i in alpha_list_tr:
    if i not in pinyinLib and len(i)>0:
        en_list.append(i)

In [ ]:
en_list

In [ ]:
dev_en_list = []
for i in dev_alpha_list:
    if i not in pinyinLib and len(i)>0:
        dev_en_list.append(i)

In [ ]:
print("英文字母：", en_list)

In [ ]:
print("英文字母：", dev_en_list)

In [ ]:
import json

with open("./pny_dict.json", "w") as fw:
    fw.write(json.dumps({"train": list(train_pny), "dev":list(dev_pny)}))

In [5]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

In [ ]:
vocab = []
line = "12个[UNK]的 小app兔子mike r n g im a   teacher ".split(' ')
for patch in line:
    if patch.encode("utf-8").isalpha():
        if patch not in vocab:
            vocab.append(patch)
    elif patch == "[UNK]":
        print(patch)
    else:
        en_str = ""
        for han in patch:
            if not is_Chinese(han):
                en_str += han
            elif is_Chinese(han):            
                if en_str != "" and en_str not in vocab:
                    vocab.append(en_str)
                    en_str = ""
                if han not in vocab:
                    vocab.append(han)
            else:
                print("ERROR:", han)
        if en_str != "" and en_str not in vocab:
            vocab.append(en_str) 
            en_str = ""
vocab

In [23]:
def mk_lm_han_vocab(data):
    vocab = ['<PAD>']
    for line in tqdm(data):
        if line in vocab:
            continue
        line = line.split(' ')
        for patch in line:
            if patch.encode("utf-8").isalpha():
                if patch not in vocab:
                    vocab.append(patch)
            elif patch == "[UNK]":
                print(patch)
            else:
                en_str = ""
                for han in patch:
                    if not is_Chinese(han):
                        en_str += han
                    elif is_Chinese(han):            
                        if en_str != "" and en_str not in vocab:
                            vocab.append(en_str)
                            en_str = ""
                        if han not in vocab:
                            vocab.append(han)
                    else:
                        print("ERROR:", han)
                if en_str != "" and en_str not in vocab:
                    vocab.append(en_str) 
                    en_str = ""
    return vocab

In [53]:
def mk_lm_han_vocab(data, vocab = ['<PAD>']):
    vocab = vocab
    # 对每句话进行分析
    for line in tqdm(data, ascii=True):
        _hlst = []
        if line in vocab: # 如果该句话在vocab中
            han_lst.append([line])
            continue
        line = line.split(' ') # 分词处理
        for patch in line:
            if patch.encode("utf-8").isalpha(): # 如果全为英文字母
                _hlst.append(patch)
                if patch not in vocab:
                    vocab.append(patch)
            elif patch == "[UNK]": # 如果等于特例[UNK]
                _hlst.append(patch)
                print(patch)
            else:               
                en_str = ""
                for han in patch:
                    if not is_Chinese(han): # 如果非汉字，就继续循环
                        en_str += han
                    elif is_Chinese(han):   # 如果是汉字，就终止en_str累加

                        if en_str != "":
                            _hlst.append(en_str)
                        _hlst.append(han)

                        if en_str != "" and en_str not in vocab:
                            vocab.append(en_str)
                        en_str = ""
                        if han not in vocab:
                            vocab.append(han)
                    else:
                        print("ERROR:", han)

                if en_str != "":
                    _hlst.append(en_str)
                if en_str != "" and en_str not in vocab:
                    vocab.append(en_str) 
                    en_str = ""
        han_lst.append(_hlst)
    return vocab

In [54]:
train_df = pd.read_csv("./dataset/dev.txt", sep="\t", header=None)

In [55]:
train_df.columns = ["wav_file", "pny", "words"]

In [56]:
han_lst = []
print('make lm hanzi vocab...')
han_vocab = mk_lm_han_vocab(train_df.words, han_vocab)

make lm hanzi vocab...


100%|############################################################################| 6085/6085 [00:00<00:00, 6373.48it/s]


In [57]:
count = 0
for i in (han_vocab):
    if not is_Chinese(i):
        print(i)
        count += 1
count

<PAD>
app
olay
p
k
b
m
a
c
e
t
j
d
z
mvp
v
low
pk
nba
logo
ok
cba
A
D
N
R
B
C
qq
vip
n
buff
solo
adc
q
o
man
out
see
sky
papi
y
s
l
iphone
x
iPhone
cctv
running
gai
rapper
freestyle
flow
popping
old
school
tv
double
kill
biu
w
f
g
oppo
boss
exo
tfboys
nine
snh
miniso
kiko
clean
bb
cc
i
cosplay
h
four
eight
mv
hi
mike
boys
video
L
angelababy
broke
kbs
part
monster
demo
sm
power
cuba
get
mate
pro
mix
if
r
two
ipad
chen
po
rain
balance
pd
feel
rose
jack
kiss
me
cf
cs
cd
dancer
bgm
game
over
bug
fpp
tpp
godv
u
vector
scar
groza
sks
mini
hold
angela
baby
yellow
red
pink
purple
vivo
max
cp
ppt
perfect
ktv
wifi
lpl
ol
lol
rw
rogue
warriors
xo
gucci
rng
royal
never
giveup
open
ng
faker
dj
amazing
CP
country
office
party
police
QQ
why
mylefolen
shy
ban
counter
pick
roll
gank
im
teacher
jasper
cpa
buy
purchase
kawasaki
polo
combo
nice
gay
one
idea
no
sorry
pos
plc
photo
shop
word
excel
adobe
fla
flash
ios
plus
xe
se
ptsd
ip
walking
dead
capcom
umbrella
ten
touch
mat
magic
hello
boos
do
dota
carr

269

In [58]:
len(han_vocab)

4101

In [47]:
train_df["han_list"] = han_lst

In [48]:
label_list = [list(i) for i in (train_df.loc[:, ["wav_file", "han_list"]].values)]

In [17]:
import json

In [59]:
with open("labels.json", "w") as f:
    f.write(json.dumps(han_vocab))

In [41]:
with open("train_index.json", "w") as f:
    f.write(json.dumps(label_list))

In [49]:
with open("dev_index.json", "w") as f:
    f.write(json.dumps(label_list))

In [67]:
with open("./dataset/dev_index.json", "r") as f:
    idx = json.load(f)

In [68]:
idx

[['./dataset/dev(SPK)/dev_byte(SPK001)/wave_1.bin',
  ['妞', '妞', '今', '天', '我', '们', '聊', '什', '么', '呢']],
 ['./dataset/dev(SPK)/dev_byte(SPK002)/wave_2.bin',
  ['我', '们', '今', '天', '聊', '动', '物']],
 ['./dataset/dev(SPK)/dev_byte(SPK001)/wave_3.bin', ['好', '吧']],
 ['./dataset/dev(SPK)/dev_byte(SPK001)/wave_5.bin',
  ['那', '你', '说', '一', '说', '动', '物', '吧']],
 ['./dataset/dev(SPK)/dev_byte(SPK002)/wave_6.bin',
  ['阿', '姨', '你', '喜', '欢', '什', '么']],
 ['./dataset/dev(SPK)/dev_byte(SPK001)/wave_7.bin', ['我', '喜', '欢', '小', '狗']],
 ['./dataset/dev(SPK)/dev_byte(SPK001)/wave_9.bin',
  ['你', '喜', '欢', '什', '么', '呀']],
 ['./dataset/dev(SPK)/dev_byte(SPK002)/wave_10.bin', ['我', '想', '问', '你']],
 ['./dataset/dev(SPK)/dev_byte(SPK001)/wave_11.bin',
  ['那', '你', '问', '我', '吧']],
 ['./dataset/dev(SPK)/dev_byte(SPK002)/wave_13.bin', ['你']],
 ['./dataset/dev(SPK)/dev_byte(SPK002)/wave_15.bin',
  ['你', '在', '动', '物', '园', '儿', '看', '过', '小', '狗', '吗']],
 ['./dataset/dev(SPK)/dev_byte(SPK001)/wave_16.

In [60]:
import tarfile

In [61]:
t = tarfile.open("./0106torch/pkg/librosa-0.7.1.tar.gz")

In [62]:
t.extractall(path = "./0106torch/pkg/")

In [ ]:
han_vocab

In [ ]:
set(dev_pny_vocab).difference(set(pny_vocab))

In [ ]:
len(set(han_vocab)),len(set(han_vocab))